In [ ]:
import re
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, BertModel
from tqdm import tqdm
import optuna
from bertviz import head_view, model_view
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, precision_recall_curve, average_precision_score, auc
import torch.nn.functional as F

In [ ]:
def load_dataset(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
    
    data = []
    current_label = None
    for line in lines:
        line = line.strip()
        if line.startswith('>'):
            label_match = re.match(r">(\d)_\d+", line)
            if label_match:
                current_label = int(label_match.group(1))
        else:
            if current_label is not None and len(line) > 0:
                data.append((current_label, line))
                current_label = None
    return pd.DataFrame(data, columns=['label', 'sequence'])

def tokenizer_kmer(sequence, k=1):
    return [sequence[i:i + k] for i in range(len(sequence) - k + 1)]